# Introduction
This notebook will be an initial look at the data that was recently gathered. 

In [28]:
import os
import glob
import pandas as pd

In [25]:
filepath = 'D:/Courses/UUJ/Research Project/masters_project/Data Collection/RawDataCollection/Closed Door'

# Change the working dir
os.chdir(filepath)

# list the contents
dir_contents = os.listdir()
# dir_contents

I have manually gone into this folder - this will need to be automated

In [27]:
# I only want the .csv files
ext = '*.csv'
files = glob.glob(ext)
files[0]

'grid1_door_closed.csv'

In [33]:
grid1_df = pd.read_csv(files[0])
grid1_df.head(10)

,[2023-01-01 15:20:48.842] Ultrasonic Sensor Data Capture Start,Unnamed: 1
0,[2023-01-01 15:20:48.845] Obstacle: closed door,NaN
1,[2023-01-01 15:20:48.845] Grid place: 1 (Steady),NaN
2,[2023-01-01 15:21:06.596] Ch1:,NaN
3,[2023-01-01 15:21:06.609] 604,NaN
4,[2023-01-01 15:21:06.610] 605,NaN
5,[2023-01-01 15:21:06.618] 605,NaN
6,[2023-01-01 15:21:06.622] 605,NaN
7,[2023-01-01 15:21:06.630] 597,NaN
8,[2023-01-01 15:21:06.634] 597,NaN
9,[2023-01-01 15:21:06.642] 597,NaN


In [38]:
# I just want the first column
col_names = grid1_df.columns
col_names[0]
data = grid1_df[col_names[0]]

In [39]:
data

0          [2023-01-01 15:20:48.845] Obstacle: closed door
1         [2023-01-01 15:20:48.845] Grid place: 1 (Steady)
2                          [2023-01-01 15:21:06.596] Ch1: 
3                            [2023-01-01 15:21:06.609] 604
4                            [2023-01-01 15:21:06.610] 605
                                ...                       
243851                       [2023-01-01 15:51:31.287] 605
243852                       [2023-01-01 15:51:31.295] 611
243853                       [2023-01-01 15:51:31.299] 611
243854                       [2023-01-01 15:51:31.307] 611
243855                       [2023-01-01 15:51:31.311] 610
Name: [2023-01-01 15:20:48.842] Ultrasonic Sensor Data Capture Start, Length: 243856, dtype: object

We can see above that there is a column with a lot of `NaN` values. I want to check is this is all this column contains. I have a feeling there is something else in that column. Actually, looking at it below, we can see the `dtype: float64` 

In [43]:
grid1_df[col_names[1]]

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
243851   NaN
243852   NaN
243853   NaN
243854   NaN
243855   NaN
Name: Unnamed: 1, Length: 243856, dtype: float64

In [52]:
nan_column = grid1_df[col_names[1]]
# type(nan_column)
nan_filtered = nan_column.isnull()

In [55]:
nan_filtered
res = [i for i, val in enumerate(nan_filtered) if not val]
res

[]

Okay, so, there is nothing in that column, so we can forget it now. We will focus on the `data` dataframe.

In [58]:
data.describe()

count                            243856
unique                           243713
top       [2023-01-01 15:50:45.920] 604
freq                                  3
Name: [2023-01-01 15:20:48.842] Ultrasonic Sensor Data Capture Start, dtype: object

Now I want to try and get the row indexs that contain the start and stop terms

In [85]:
# data.str.contains("start")
ch1_start_idx = [i for i,x in enumerate(data.str.contains("Ch1:")) if x]
ch2_start_idx = [i for i,x in enumerate(data.str.contains("Ch2:")) if x]
print("Channel 1 start indexes are:", ch1_start_idx)
print("Channel 2 start indexes are:", ch2_start_idx)

Channel 1 start indexes are: [2, 200012]
Channel 2 start indexes are: [100007]


Now, I will want to get the ending indexes.

In [84]:
ch1_end_idx = [i for i,x in enumerate(data.str.contains("Ch1 End")) if x]
ch2_end_idx = [i for i,x in enumerate(data.str.contains("Ch2 End")) if x]
print("Channel 1 end indexes are:", ch1_end_idx)
print("Channel 2 end indexes are:", ch2_end_idx)

Channel 1 end indexes are: [100004]
Channel 2 end indexes are: [200009]


Now, I want to get the times.

In [86]:
ch1_time_idx = [i for i,x in enumerate(data.str.contains("Channel 1 time")) if x]
ch2_time_idx = [i for i,x in enumerate(data.str.contains("Channel 2 time")) if x]
print("Channel 1 time indexes is:", ch1_time_idx)
print("Channel 2 time indexes is:", ch2_time_idx)

Channel 1 time indexes is: [100005]
Channel 2 time indexes is: [200010]


Now I need to slice up my data to remove the timestamp and turn the string formatted number into an int

In [103]:
str_numbers = data.str[-4:]
ch1_str_nums = str_numbers.iloc[3:] # This is the actual distance numbers